<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

wandb.init(project="assignment-1", entity="swathi")

classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  return {
      "x_train": x_train,
      "y_train": y_train,
      "x_test": x_test,
      "y_test": y_test
  }

def log_images():
  dataset = load_data()
  train_images = dataset["x_train"]
  train_labels = dataset["y_train"]
  images = []
  labels = []
  for i in range(len(train_images)):
    if len(labels)==len(classes):
      break
    if classes[train_labels[i]] not in labels:
      images.append(train_images[i])
      labels.append(classes[train_labels[i]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

def dataset():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  return{
      'train':{
          'x': x_train.reshape([60000,784]),
          'y': y_train.reshape([60000,1])
      },
      'test':{
          'x': x_test.reshape([10000,784]),
          'y': y_test.reshape([10000,1])
      }
  }

load_data()
dataset()
log_images()

_runtime,4
_timestamp,1614727702
_step,0


_runtime,▁
_timestamp,▁
_step,▁


Weights initialising

In [ ]:
#l=no_of_layers
N= np.zeros((L,1)) # Number of neurons in each layer
N[0]=X_input.shape[2]
W = []
grad_W=[]
b = []
grad_b=[]
a=[]
grad_a=[]
a.append(0)
h=[]
grad_h=[]
h.append(X_input)

np.random.seed(0)
 


In [ ]:
def initialize_weights(N):
  W.append(0)
  b.append(0)
  for i in range(1,L+1):
    W.append(np.random.randn(N[i],N[i-1]))
    b.append(np.random.randn(N[1],1))

  return W,b
  

In [ ]:
def one_hot(y):
  one_hot_y=[]
  for i in range(N[0]):
    one_hot=np.zeros((10,1))
    one_hot[y[i]-1] = 1
    one_hot_y.append(one_hot)
  
  return one_hot_y

Softmax

In [ ]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [ ]:
def sigmoid(a):

  return 1/(1+np.exp(-a))


Sigmoid derivative

In [ ]:
def dSigmoid(a):

  return sigmoid(a)*(1-sigmoid(a))

Feed forward network

In [ ]:
def feedforward(X_input, W, b, N, l):
    h_prev=X_input
  for i in range(1,l) :
    a_new=b[i]+np.matmul(W[i],h_prev)
    h=sigmoid(a_new)
    a.append(a_new)
    h.append(h)
    h_prev=h

  a_new=b[l]+np.matmul(W[l],h_prev)
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [ ]:
def backrop(y,y_hat,W,a,h):
  da = y_hat - one_hot(y)
  grad_a.append(da)

  for i in reversed(range(1,L+1)):
    dw = np.matmul(da,np.transpose(h[i-1]))
    db = da
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dSigmoid(a[i-1])
    
    grad_W.append(dw)
    grad_b.append(db)
    grad_a.append(da_prev)
    grad_h.append(dh_prev)

  return grad_W,grad_b,grad_a,grad_h


Gradient Descent

In [ ]:
def grad_descent(N,l,X_input,y,opt):
  if opt == 'sgd' :
    W = W - grad_W
    b = b - grad_b

  if opt == 'mom':
    


Training

In [ ]:
def training(N,l,opt,epochs,X_input,y):
  
  W , b = initialize_weights(N)
  for i in range(epochs):
    a , h , y_hat = feedforward(X_input, W, b, N, l)  
    grad_W , grad_b , ~ , ~ = backrop(y,y_hat,W,a,h)
    W , b = grad_descent()

  return W,b


In [ ]:
def predict(W,b,X_test,N,l):
  ~ , ~ , y_hat = feedforward(X_test, W, b, N, l)
  y_class = np.argmax(y_hat,axis=0)

  return y_class